In [ ]:
import requests
from requests_oauthlib import OAuth1
import os
import time
# import caffeine
# caffeine.off()

# Construct a "consumer" from the key/secret provided by MediaWiki
from configEiabot import consumer_key, consumer_secret, access_token, access_secret  # You'll need to provide this
# from configEffeietsanders import consumer_key, consumer_secret, access_token, access_secret  # You'll need to provide this

OLDCOMMENT = "new data"

In [ ]:
auth = OAuth1(consumer_key, consumer_secret, access_token, access_secret)
sandboxpage = "User:Effeietsanders/Sandbox"

# requests.post(url='https://en.wikipedia.org/w/api.php', data=data, auth=auth)
new_date = "6 December 2020"

In [ ]:
print("Reading top 10 watchlist items")
response = requests.get(
    "https://commons.wikimedia.org/w/api.php",
    params={
        'action': "query",
        'list': "watchlist",
        'wllimit': 10,
        'wlprop': "title|comment",
        'format': "json"
    },
    auth=auth
)
for item in response.json()['query']['watchlist']:
    print("{title}\t{comment}".format(**item))

In [ ]:
URL = "https://commons.wikimedia.org/w/api.php"
# FILE_NAME = "COVID-19_Prevalence_in_Alabama_by_county.svg"
# FILE_PATH = 'output/' + FILE_NAME
COMMENT = 'new data: ' + new_date
# COMMENT = 'data from 16 July 2020, but with two color levels added for rolling 14-day prevalence'

In [ ]:
def getCSRF_Token (URL = "https://commons.wikimedia.org/w/api.php", auth = auth):
    # Step 3: Obtain a CSRF token
    PARAMS_3 = {
        "action": "query",
        "meta":"tokens",
        "format":"json"
    }

    R = requests.get(url=URL, params=PARAMS_3, auth=auth)
    print("Get token: ", R)
    DATA = R.json()
    if 'query' in DATA.keys():
        CSRF_TOKEN = DATA["query"]["tokens"]["csrftoken"]
    else:
        print(DATA)
    return(CSRF_TOKEN)

In [ ]:
def uploadFile (CSRF_TOKEN,
                auth = auth,
                FILE_NAME = "COVID-19_Prevalence_in_Alabama_by_county.svg",
                FILE_PATH = 'output/COVID-19_Prevalence_in_Alabama_by_county.svg',
                COMMENT = "new data",
                URL = "https://commons.wikimedia.org/w/api.php"
                ) :
    PARAMS_4 = {
        "action": "upload",
        "filename": FILE_NAME,
        "comment": COMMENT,
        "format": "json",
        "token": CSRF_TOKEN,
        "ignorewarnings": 1,
    }
    FILE = {'file':(FILE_NAME, open(FILE_PATH, 'rb'), 'multipart/form-data')}
    R = requests.post(URL, files=FILE, data=PARAMS_4, auth=auth)
    DATA = R.json()
    if 'upload' in DATA.keys():
        print("success", " for ", FILE_NAME)
    elif 'error' in DATA.keys():
        if 'code' in DATA['error'].keys():
            print("error for: ", FILE_NAME, "\n", DATA["error"]["code"])
        else:
            print("error for: ", FILE_NAME, "\n", DATA["error"])
    else:
        print(FILE_NAME, ": \n", DATA)
    return(DATA)


In [ ]:
def editPage (CSRF_TOKEN,
              newdate,
             auth = auth,
             PAGE_NAME = sandboxpage, #"Template:COVID-19_Deaths_Prevalence_in_US_by_county",
             URL = "https://commons.wikimedia.org/w/api.php",
              start = "<!-- date -->",
              end = "<!-- /date -->"
             ):
    # First, get the page content
    # https://www.mediawiki.org/wiki/API:Get_the_contents_of_a_page
    
    PARAMS_parse = {
        "action": "parse",
        "page": PAGE_NAME,
        "format": "json",
        "prop": "wikitext"
    }

    R = requests.get(url=URL, params=PARAMS_parse)
    DATA_parse = R.json()
    if 'parse' in DATA_parse.keys():
            # Make the change
        pagetext = DATA_parse["parse"]["wikitext"]["*"]
#         print(pagetext)
        pagetext2 = pagetext.split(start)[0] + start + newdate + end + pagetext.split(end)[1]
        print("loaded: ", PAGE_NAME)
    else:
        print(DATA_parse)
        throw("error!")
        
    # Based on https://www.mediawiki.org/wiki/API:Edit
    PARAMS_edit = {
        "action": "edit",
        "title": PAGE_NAME,
        "token": CSRF_TOKEN,
        "format": "json",
        "text": pagetext2,
        "summary": "new date: " + newdate
    }
    R = requests.post(URL, data=PARAMS_edit, auth=auth)
    DATA_edit = R.json()
    print(R, DATA_edit)


In [ ]:
def timeawait(waittime = 10):
    time.sleep(-time.time()%waittime) #only upload once every X seconds

In [ ]:
script_dir = os.path.dirname(os.path.realpath('maps'))
folderPath = 'output'
print(script_dir)

In [ ]:
file_list = []
for base_path, dir_names, file_names in os.walk(script_dir + '/' + folderPath):
    for file_name in file_names:
        if file_name.endswith((".svg", ".gif")):
            if file_name.startswith(("COVID-19_Deaths_Prevalence_in_", 
                                     "COVID-19_Prevalence_in_",
                                     "COVID-19_rolling_14day_Prevalence_in_")):
                file_path = os.path.join(base_path, file_name)
                rel_path = os.path.relpath(file_path, script_dir)
                file_list.append([rel_path, file_name, False])

In [ ]:
# caffeine.on(display=True)
print("starting time", time.time())
if COMMENT == OLDCOMMENT:
    print("update your comment!")
else:
    outputs = []
    CsrfToken = getCSRF_Token(URL = "https://commons.wikimedia.org/w/api.php", auth = auth)
    for file in file_list:
        if file[2] == False:
            uploaded = uploadFile(CSRF_TOKEN = CsrfToken,
                      auth = auth,
                      FILE_NAME= file[1],
                      FILE_PATH= file[0],
                      COMMENT = COMMENT,
                      URL = URL)
            outputs.append(uploaded)
            timeawait(waittime=6) #only upload once every 10s
            file[2] = True
        elif file[2] == True: #already uploaded these files
            print("skipping ", file[0])
        else:
            print("error: ", file)
            break
    OLDCOMMENT = COMMENT #make sure we don't forget to update the comment next time
    print("done! ", time.time())

In [ ]:
CsrfToken = getCSRF_Token(URL = "https://commons.wikimedia.org/w/api.php", auth = auth)
pagelist = [
    "Template:COVID-19_Deaths_Prevalence_in_US_by_county",
    "Template:COVID-19 rolling 14-day Prevalence in US by county",
    "Template:COVID-19 Prevalence in US by county"
]
for page in pagelist:
    timeawait(waittime=10)
    editPage(CSRF_TOKEN = CsrfToken, 
             newdate = new_date, 
             auth=auth,
             PAGE_NAME = page,
            )
# caffeine.off()
# print(caffeine.verify())

In [ ]:
##
# print(caffeine.verify())

In [ ]:
break

In [ ]:
outputs = []
CsrfToken = getCSRF_Token(URL = "https://commons.wikimedia.org/w/api.php", auth = auth)
for file in [['output/COVID-19_Prevalence_in_Massachusetts_by_county.svg', 'COVID-19_Prevalence_in_Massachusetts_by_county.svg'],
            ['output/COVID-19_rolling_14day_Prevalence_in_Massachusetts_by_county.svg', 'COVID-19_rolling_14day_Prevalence_in_Massachusetts_by_county.svg'],
            ['output/COVID-19_Deaths_Prevalence_in_Massachusetts_by_county.svg', 'COVID-19_Deaths_Prevalence_in_Massachusetts_by_county.svg']]:
    uploaded = uploadFile(CSRF_TOKEN = CsrfToken,
              auth = auth,
              FILE_NAME= file[1],
              FILE_PATH= file[0],
              COMMENT = 'bug fix Massachusetts',
              URL = URL)
    outputs.append(uploaded)
    time.sleep(-time.time()%10) #only upload once every 10s
OLDCOMMENT = COMMENT #make sure we don't forget to update the comment next time
print("done!")
    
outputs[-1]

In [ ]:
OLDCOMMENT = "new data"

In [ ]:
# file_list_AL = [ ['output/COVID-19_Prevalence_in_Alabama_by_county.svg',
#   'COVID-19_Prevalence_in_Alabama_by_county.svg'],]

In [ ]:
CsrfToken = getCSRF_Token(URL = "https://en.wikipedia.org/w/api.php", auth = auth)
pagelist = [
    "User:Effeietsanders/Sandbox"
]
for page in pagelist:
    timeawait(waittime=10)
    editPage(CSRF_TOKEN = CsrfToken, 
             newdate = new_date, 
             auth=auth,
             PAGE_NAME = page,
            )

In [ ]:
outputs[-1]